<a href="https://colab.research.google.com/github/hadiroudsari/wine-detection/blob/master/wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Country  of Wine prediction
This project was prepered for bigdata course final project. I have done some task on this specific dataset.  
From our exploratory data analysis below, we will observed:
- Cleaning the dataset 
- shuffling the data and sample it because the original data set is 129975 row.
- Data pre processing  of the descripyion for NLP;
- Feature engineering using tf-idf to get numeric value from terms_stemmed column.
- Encoding the category column country to index
- Training the Multilayer perceptron classifier model
- Training the One-vs-Rest classifier.



here i load JDK 8 LTS


---



In [ ]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64" # Set path to JAVA_HOME


## **1.** Install PySpark and related dependencies

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

openjdk-8-jdk-headless is already the newest version (8u252-b09-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


## **2.** Import useful PySpark packages *italicized text*

In [ ]:
import requests
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf



# **3.** Create Spark context

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
sc._conf.getAll()

[('spark.driver.memory', '45G'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1592590277045'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.port', '36137'),
 ('spark.ui.port', '4050'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.memory', '4G'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'dc6ccb28d348')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Link Google Colab to our Google Drive**

In [ ]:
GDRIVE_DIR = "/content/gdrive" # Your own mount point on Google Drive
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive" # Your own home directory
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/1898985/dataSet" # Your own data directory

## **1. Link Google Colab to our Google Drive**

In [ ]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/gdrive


## **2. Data Acquisition**

### **Read dataset file into a Spark Dataframe**


In [ ]:
wine_df = spark.read.load(GDRIVE_DATA_DIR, 
                           format="csv", 
                           sep=",", 
                           inferSchema="true", 
                           header="true")

### **get the overview of dataset rows and columns**

In [ ]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(wine_df.count(), len(wine_df.columns)))

The shape of the dataset is 129975 rows by 14 columns


In [ ]:
wine_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



*convert pyspark dataframe to panda dataset to shuffle on panda dataset and then turn it back to pyspark dataframe.*

In [ ]:

p_wine_df = wine_df.toPandas()
p_wine_df=p_wine_df.sample(frac=1)
p_wine_df = p_wine_df.iloc[98000:,]
wine_df=spark.createDataFrame(p_wine_df)
print("The shape of the dataset is {:d} rows by {:d} columns".format(wine_df.count(), len(wine_df.columns)))

The shape of the dataset is 31975 rows by 14 columns



**Under sample 29975 row from 129975 rows.**
 

- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- Cleaning the dataset 
- Data pre processing  of the descripyion for NLP;
- Feature engineering using tf-idf to get numeric value from terms_stemmed column.
- Encoding the category column country to index
- Training the Multilayer perceptron classifier model
- Training the One-vs-Rest classifier.

# **Cleaing the data set**

Count the number of duplicated description (if any)

In [ ]:
print("The total number of duplicated description are {:d} out of {:d}".
      format(wine_df.count() - wine_df.dropDuplicates(['description']).count(), wine_df.count()))

The total number of duplicated description are 573 out of 31975


### **Remove duplicate description**

In [ ]:
wine_df = wine_df.dropDuplicates(["description"])

In [ ]:
print("The total number of unique description is: {:d}".format(wine_df.count()))

The total number of unique description is: 31402


### Check for any missing value (i.e., <code>NULL</code>) along <code>description</code> and country column and Removing the rows that have null description and null country.

In [ ]:
wine_df = wine_df.na.drop(subset=["country"])
wine_df.where(col("country").isNull()).count()

wine_df = wine_df.na.drop(subset=["description"])
wine_df.where(col("description").isNull()).count()

0

## The numbers of  Wine's country 

In [ ]:
wine_df.dropDuplicates(['country']).count()

38

**A bit Data exploration**

In [ ]:
 for c in wine_df.columns:
  print("N. of missing values of column `{:s}` = {:d}".format(c, wine_df.where(col(c).isNull()).count())) 

N. of missing values of column `_c0` = 0
N. of missing values of column `country` = 0
N. of missing values of column `description` = 0
N. of missing values of column `designation` = 9031
N. of missing values of column `points` = 0
N. of missing values of column `price` = 2153
N. of missing values of column `province` = 0
N. of missing values of column `region_1` = 5015
N. of missing values of column `region_2` = 19101
N. of missing values of column `taster_name` = 6253
N. of missing values of column `taster_twitter_handle` = 7433
N. of missing values of column `title` = 1
N. of missing values of column `variety` = 1
N. of missing values of column `winery` = 2


There is a lot missing value from <code>designation</code>,<code>region_1</code>,<code> region 2 </code>,<code>taster_name</code>,<code>taster_twitter_handle</code> and <code>price</code> columns, so we will drop these columns.also the taster rwitter is not valuable data for us anyway.and in this traing we want to predict the country origin of wine from description, so we just need the description column and country column.

In [ ]:
columns_to_drop = ['designation', 'region_1','region_2','taster_name','taster_twitter_handle','points','price','province','title','variety','winery']
wine_df=wine_df.drop(*columns_to_drop)
wine_df.show(5)

+------+--------+--------------------+
|   _c0| country|         description|
+------+--------+--------------------+
| 57759|  France|89-91. Barrel sam...|
|108609|  France|92–94. Barrel sam...|
| 85318|      US|A bit difficult t...|
| 62173|Portugal|A blend of Alvari...|
| 94149|  Israel|A co-fermented bl...|
+------+--------+--------------------+
only showing top 5 rows



- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- ~Cleaning the dataset~. (**DONE**)
- Data pre processing  of the descripyion for NLP;
- Feature engineering using tf-idf to get numeric value from terms_stemmed column.
- Encoding the category column country to index
- Training the Multilayer perceptron classifier model
- Trrainin the One-vs-Rest classifie.

# Data Preprocessing
in this section I want to use description column that is text we know that for using text data we need to do some preprocessing such as `Text` `cleaning`,`Tokenization`,`Stopwords`, `removal` and `Stemming`.
These are the basic of NLP task.

In [ ]:
def clean_text(df, column_name="description"):
    """ 
    This fucntion takes the raw text data and apply a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_description` column as the results of the NLP preprocessing pipeline

    """
    from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
    from pyspark.ml.feature import Tokenizer, StopWordsRemover
    from nltk.stem.snowball import SnowballStemmer

    # Text preprocessing pipeline
    print("***** Text Preprocessing Pipeline *****\n")

    # 1. Text cleaning
    print("# 1. Text Cleaning\n")
    # 1.a Case normalization
    print("1.a Case normalization:")
    lower_case_news_df = df.select("_c0","country", lower(col(column_name)).alias(column_name))
    #lower_case_news_df.show(10,truncate=False)
    # 1.b Trimming
    print("1.b Trimming:")
    trimmed_news_df = lower_case_news_df.select("_c0","country", trim(col(column_name)).alias(column_name))
    # trimmed_news_df.show(10,truncate=False)
    # 1.c Filter out punctuation symbols
    print("1.c Filter out punctuation:")
    no_punct_news_df = trimmed_news_df.select("_c0","country", (regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name))
    no_punct_news_df.show(10)
    # 1.d Filter out any internal extra whitespace
    print("1.d Filter out extra whitespaces:")
    cleaned_news_df = no_punct_news_df.select("_c0","country", trim(regexp_replace(col(column_name), " +", " ")).alias(column_name))
    # cleaned_news_df.show(10)

    # 2. Tokenization (split text into tokens)
    print("# 2. Tokenization:")
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_news_df).cache()
    # tokens_df.show(10)

    # 3. Stopwords removal
    print("# 3. Stopwords removal:")
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    terms_df = stopwords_remover.transform(tokens_df).cache()
    # terms_df.show(10)

    # 4. Stemming (Snowball stemmer)
    print("# 4. Stemming:")
    stemmer = SnowballStemmer(language="english")
    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
    terms_stemmed_df = terms_df.withColumn("terms_stemmed", stemmer_udf("terms"))
    terms_stemmed_df.show(10)
    
    return terms_stemmed_df.cache()

In [ ]:
clean_wine_description_df = clean_text(wine_df)

***** Text Preprocessing Pipeline *****

# 1. Text Cleaning

1.a Case normalization:
1.b Trimming:
1.c Filter out punctuation:
+------+--------+--------------------+
|   _c0| country|         description|
+------+--------+--------------------+
| 57759|  France| barrel sample th...|
|108609|  France| barrel sample ti...|
| 85318|      US|a bit difficult t...|
| 62173|Portugal|a blend of alvari...|
| 94149|  Israel|a cofermented ble...|
| 27614|  France|a delightful blen...|
|126433|      US|a distinctive win...|
| 25798|      US|a forestlike eart...|
|100855|  France|a fresh pale ros ...|
| 64825|      US|a modern popular ...|
+------+--------+--------------------+
only showing top 10 rows

1.d Filter out extra whitespaces:
# 2. Tokenization:
# 3. Stopwords removal:
# 4. Stemming:
+------+--------+--------------------+--------------------+--------------------+--------------------+
|   _c0| country|         description|              tokens|               terms|       terms_stemmed|
+----

- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- ~Cleaning the dataset~. (**DONE**)
- ~Data pre processing  of the descripyion for NLP~.(**DONE**)
- Feature engineering using tf-idf to get numeric value from terms_stemmed column.
- Encoding the category column country to index
- Training the Multilayer perceptron classifier model
- Trrainin the One-vs-Rest classifie.

# TF-IDF 

tf-idf (term frequency-inverse document frequency) is for some how making our words text document into numeric value this is some how feature enginering because we can not use our alphabetic features into our models.

In [ ]:
def extract_tfidf_features(df, column_name="terms_stemmed"):
    """ 
    This fucntion takes the text data and converts it into a term frequency-inverse document frequency vector

    parameter: dataframe
    returns: dataframe with tf-idf vectors

    """

    # Importing the feature transformation classes for doing TF-IDF 
    from pyspark.ml.feature import HashingTF, CountVectorizer, IDF
    from pyspark.ml import Pipeline


    # USING PIPELINE
    cv = CountVectorizer(inputCol=column_name, outputCol="tf_features", vocabSize=100, minDF=8)
    # hashingTF = HashingTF(inputCol=column_name, outputCol="tf_features", numFeatures=100)
    idf = IDF(inputCol="tf_features", outputCol="features")

    pipeline = Pipeline(stages=[cv, idf]) # replace `cv` with `hashingTF` if needed
    features = pipeline.fit(df)
    tf_idf_features_df = features.transform(df).cache()

    return tf_idf_features_df

In [ ]:
tf_idf_df = extract_tfidf_features(clean_wine_description_df)

In [ ]:
tf_idf_df.select(col("features")).show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(100,[0,2,5,7,32,46,63,73,78],[0.6802593024878884,0.9093669920889772,1.210377992987827,1.4055043731493984,2.275142190192505,2.545913790495969,2.771091930507967,2.910752045073863,2.966740183765621])    

- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- ~Cleaning the dataset~. (**DONE**)
- ~Data pre processing  of the descripyion for NLP~.(**DONE**)
- ~Feature engineering using tf-idf to get numeric value from terms_stemmed column~.(**DONE**)
- Encoding the category column country to index
- Training the Multilayer perceptron classifier model
- Trrainin the One-vs-Rest classifie.

In [ ]:
columns_to_drop = ['description', 'tokens','terms','terms_stemmed','tf_features']
tf_idf_df=tf_idf_df.drop(*columns_to_drop)
tf_idf_df.show(5)

+------+--------+--------------------+
|   _c0| country|            features|
+------+--------+--------------------+
| 57759|  France|(100,[0,2,5,7,32,...|
|108609|  France|(100,[0,2,7,11,37...|
| 85318|      US|(100,[0,7,8,25,28...|
| 62173|Portugal|(100,[2,5,14,16,2...|
| 94149|  Israel|(100,[0,1,2,4,7,9...|
+------+--------+--------------------+
only showing top 5 rows



cheack and remove any possible zero length vector  

In [ ]:
@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

In [ ]:
print("Total n. of zero-length vectors: {:d}".
      format(tf_idf_df.where(num_nonzeros("features") == 0).count()))

Total n. of zero-length vectors: 8


removing zero length

In [ ]:
tf_idf_df = tf_idf_df.where(num_nonzeros("features") > 0)

In [ ]:
print("Total n. of zero-length vectors (after removal): {:d}".
      format(tf_idf_df.where(num_nonzeros("features") == 0).count()))

Total n. of zero-length vectors (after removal): 0


for neural network training we need to our label data be indexed. so I use `StringIndexer`.




In [ ]:
from pyspark.ml.feature import StringIndexer


indexer = StringIndexer(inputCol="country", outputCol="countryIndex")
tf_idf_df_index = indexer.fit(tf_idf_df).transform(tf_idf_df)



In [ ]:
tf_idf_df_index.show(5)

+------+--------+--------------------+------------+
|   _c0| country|            features|countryIndex|
+------+--------+--------------------+------------+
| 57759|  France|(100,[0,2,5,7,32,...|         1.0|
|108609|  France|(100,[0,2,7,11,37...|         1.0|
| 85318|      US|(100,[0,7,8,25,28...|         0.0|
| 62173|Portugal|(100,[2,5,14,16,2...|         4.0|
| 94149|  Israel|(100,[0,1,2,4,7,9...|        13.0|
+------+--------+--------------------+------------+
only showing top 5 rows



- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- ~Cleaning the dataset~. (**DONE**)
- ~Data pre processing  of the descripyion for NLP~.(**DONE**)
- ~Feature engineering using tf-idf to get numeric value from terms_stemmed column~.(**DONE**)
- 
~Encoding the category column country to index.~(**DONE**)
- Training the Multilayer perceptron classifier model
- Training the One-vs-Rest classifier.

In [ ]:
# Randomly split our original dataset `house_df` into 90÷10 for training and test, respectively
train_df, test_df = tf_idf_df_index.randomSplit([0.8, 0.2], seed=1234)

# Multilayer perceptron classifier

In [ ]:
train_df.dropDuplicates(['countryIndex']).count()

38

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [100, 130, 50, train_df.dropDuplicates(['countryIndex']).count()]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234,featuresCol='features', labelCol='countryIndex')
# train the model
model = trainer.fit(train_df)

In [ ]:
#compute accuracy on the test set
result = model.transform(test_df)
predictionAndLabels = result.select("prediction", "countryIndex")
predictionAndLabels = predictionAndLabels.withColumnRenamed("countryIndex", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")


In [ ]:
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.6751761942051684


In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName="f1")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.6518270753025871


- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- ~Cleaning the dataset~. (**DONE**)
- ~Data pre processing  of the descripyion for NLP~.(**DONE**)
- ~Feature engineering using tf-idf to get numeric value from terms_stemmed column~.(**DONE**)
- 
~Encoding the category column country to index.~(**DONE**)
- ~Training the Multilayer perceptron classifier model~.(**DONE**)
- Training the One-vs-Rest classifier.

# One-vs-Rest classifier

In [ ]:
train_df = train_df.withColumnRenamed("countryIndex", "label")

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(train_df)


In [ ]:
test_df = test_df.withColumnRenamed("countryIndex", "label")

In [ ]:
predictions = ovrModel.transform(test_df)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.410765


- ~Shuffling the data and sample it because the original thata set is 129975 row~.(**DONE**)

- ~Cleaning the dataset~. (**DONE**)
- ~Data pre processing  of the descripyion for NLP~.(**DONE**)
- ~Feature engineering using tf-idf to get numeric value from terms_stemmed column~.(**DONE**)
- 
~Encoding the category column country to index.~(**DONE**)
- ~Training the Multilayer perceptron classifier model~.(**DONE**)
- ~Trraining the One-vs-Rest classifier~.(**DONE**)